# Évaluation d'un système de recommandation My Content

Notebook pour entraîner et comparer plusieurs approches de recommandation sur le dataset Kaggle **news-portal-user-interactions-by-globocom**. L'objectif est de montrer clairement chaque étape (du chargement des données jusqu'au choix final du modèle).

In [161]:
# Imports & Config
from __future__ import annotations
import json
import os
import pickle
import time
from pathlib import Path
from typing import Callable, Dict, List, Optional, Tuple, Union
import optuna

import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

# Configuration
CONFIG = {
    "clicks_dir": "../data/news-portal-user-interactions-by-globocom/clicks",
    "metadata_path": "../data/news-portal-user-interactions-by-globocom/articles_metadata.csv",
    "embeddings_path": "../data/news-portal-user-interactions-by-globocom/articles_embeddings.pickle",
    "max_click_files": 12,
    "artifacts_dir": "../artifacts/evaluation",
    "k": 5,
    "train_ratio": 0.8,
    "recent_window_days": 7,
    "random_seed": 42,
    "svd_components": 64,
    "content_pca_components": None,
    "covisit_top_n_neighbors": 100,
    "covisit_similarity": "cosine",
    "covisit_hybrid_alpha": 0.7,
    "svd_hazard_ndcg": 0.02,
}
np.random.seed(CONFIG["random_seed"])
Path(CONFIG["artifacts_dir"]).mkdir(parents=True, exist_ok=True)
print("Config ready", CONFIG)


Config ready {'clicks_dir': '../data/news-portal-user-interactions-by-globocom/clicks', 'metadata_path': '../data/news-portal-user-interactions-by-globocom/articles_metadata.csv', 'embeddings_path': '../data/news-portal-user-interactions-by-globocom/articles_embeddings.pickle', 'max_click_files': 12, 'artifacts_dir': '../artifacts/evaluation', 'k': 5, 'train_ratio': 0.8, 'recent_window_days': 7, 'random_seed': 42, 'svd_components': 64, 'content_pca_components': None, 'covisit_top_n_neighbors': 100, 'covisit_similarity': 'cosine', 'covisit_hybrid_alpha': 0.7, 'svd_hazard_ndcg': 0.02}


## Contexte

Nous voulons proposer à chaque lecteur un Top-5 d'articles susceptibles de l'intéresser. Le notebook illustre la démarche de A à Z : préparation des données, construction de différentes familles de modèles puis comparaison à l'aide de métriques de ranking.

## Données

Les fichiers attendus sont situés dans `/data/*`.

In [162]:
# Load data utilities

def detect_timestamp_column(df: pd.DataFrame) -> str:
    """Detect the timestamp-like column name."""
    candidates = ["click_timestamp", "timestamp", "event_time", "ts", "time"]
    for col in df.columns:
        if col in candidates or col.lower() in candidates:
            return col
    raise ValueError("No timestamp-like column found. Expected one of: " + ",".join(candidates))


def detect_article_column(df: pd.DataFrame) -> str:
    """Detect the article/item column name."""
    candidates = ["click_article_id", "clicked_article_id", "article_id", "item_id", "content_id"]
    for col in df.columns:
        if col in candidates:
            return col
    raise ValueError("No article id column found. Expected one of: " + ",".join(candidates))


def infer_unix_unit(values: pd.Series) -> str:
    numeric = pd.to_numeric(values, errors="coerce").dropna()
    if numeric.empty:
        return "s"
    max_abs = numeric.abs().max()
    if max_abs >= 1e14:
        return "ns"
    if max_abs >= 1e11:
        return "ms"
    return "s"


def to_timestamp(series: pd.Series) -> pd.Series:
    if pd.api.types.is_datetime64_any_dtype(series):
        return pd.to_datetime(series)
    if pd.api.types.is_numeric_dtype(series):
        unit = infer_unix_unit(series)
        return pd.to_datetime(series, unit=unit, errors="coerce")

    converted = pd.to_datetime(series, errors="coerce")
    if converted.notna().any():
        return converted

    unit = infer_unix_unit(series)
    return pd.to_datetime(series, unit=unit, errors="coerce")


def list_click_files(path: Union[str, Path]) -> List[Path]:
    path_obj = Path(path)
    if path_obj.is_file():
        return [path_obj]
    if path_obj.is_dir():
        return sorted(path_obj.glob("clicks_hour_*.csv"))
    return []


def create_synthetic_clicks(path: str, n_users: int = 50, n_items: int = 120, days: int = 30, interactions_per_user: int = 25) -> pd.DataFrame:
    """Create a small synthetic clicks dataset to keep the notebook runnable."""
    rng = np.random.default_rng(CONFIG["random_seed"])
    start = pd.Timestamp("2022-01-01")
    records = []
    for user in range(1, n_users + 1):
        offsets = rng.integers(0, days, size=interactions_per_user)
        timestamps = [start + pd.Timedelta(int(o), unit="D") for o in sorted(offsets.tolist())]
        articles = rng.integers(1, n_items + 1, size=interactions_per_user)
        for ts, art in zip(timestamps, articles):
            records.append({"user_id": int(user), "article_id": int(art), "timestamp": ts})
    df = pd.DataFrame(records).sort_values("timestamp").reset_index(drop=True)
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(path, index=False)
    print(
        f"Synthetic clicks dataset created at {path} "
        f"(users={n_users}, items={n_items}, interactions={len(df)})"
    )
    return df


def load_clicks(path: str, max_files: Optional[int] = None) -> pd.DataFrame:
    """Load clicks data from the Globo hourly files, with a safety cap."""
    files = list_click_files(path)
    if not files:
        print(f"Clicks directory not found at {path}. Generating a synthetic sample for demonstration.")
        return create_synthetic_clicks(Path(path) / "clicks_hour_000.csv")

    if max_files is not None:
        files = files[:max_files]

    frames = []
    for file in files:
        df = pd.read_csv(file)
        ts_col = detect_timestamp_column(df)
        article_col = detect_article_column(df)
        df[ts_col] = to_timestamp(df[ts_col])
        df = df.rename(columns={ts_col: "timestamp", article_col: "article_id"})
        frames.append(df[["user_id", "article_id", "timestamp"]])

    combined = pd.concat(frames, ignore_index=True)
    return combined.sort_values("timestamp").reset_index(drop=True)


def load_metadata(path: str) -> Optional[pd.DataFrame]:
    """Load article metadata if available."""
    if not os.path.exists(path):
        print(f"Metadata file not found at {path}. Falling back to co-visitation content model.")
        return None
    meta = pd.read_csv(path)
    if "article_id" not in meta.columns:
        print("Metadata missing 'article_id' column. Ignoring metadata.")
        return None
    return meta


clicks = load_clicks(CONFIG["clicks_dir"], max_files=CONFIG["max_click_files"])
metadata = load_metadata(CONFIG["metadata_path"])
print(clicks.head())
print("Metadata loaded:", metadata is not None)


   user_id  article_id               timestamp
0       59      234853 2017-10-01 03:00:00.026
1       79      159359 2017-10-01 03:00:01.702
2      154       96663 2017-10-01 03:00:04.207
3      111      202436 2017-10-01 03:00:14.140
4       70      119592 2017-10-01 03:00:18.863
Metadata loaded: True


## Analyse exploratoire des données

Courte photographie des fichiers sources immédiatement après le chargement :
- nombre de lignes et noms de colonnes des clics
- volumes et intégrité des métadonnées articles
- dimensions et structure du fichier d'`articles_embeddings`.

In [163]:
# EDA rapide sur les données sources
import pickle
from pathlib import Path
from collections.abc import Mapping


def summarize_timestamps(series: pd.Series):
    series = pd.to_datetime(series)
    daily = series.dt.date.value_counts().sort_index().rename_axis("date").reset_index(name="nb_clicks")
    hourly = series.dt.hour.value_counts().sort_index().rename_axis("hour").reset_index(name="nb_clicks")
    return series.min(), series.max(), daily, hourly


def describe_structure(obj, prefix="embeddings", max_depth=4):
    entries = []

    def add_entry(path, value, note=None):
        entry = {"chemin": path, "type": type(value).__name__}
        if hasattr(value, "shape"):
            entry["shape"] = tuple(getattr(value, "shape"))
        elif hasattr(value, "__len__") and not isinstance(value, (str, bytes)):
            entry["len"] = len(value)
        if hasattr(value, "dtype"):
            entry["dtype"] = str(getattr(value, "dtype"))
        if note:
            entry["note"] = note
        if isinstance(value, np.ndarray) and value.dtype.names:
            entry["dtype_fields"] = list(value.dtype.names)
        if isinstance(value, np.ndarray) and value.ndim == 1 and len(value) > 0 and not isinstance(value[0], (np.ndarray, list, tuple, Mapping)):
            entry["exemple"] = repr(value[:3].tolist())
        entries.append(entry)

    def walk(value, path, depth):
        add_entry(path, value)
        if depth >= max_depth:
            return
        if isinstance(value, Mapping):
            for k, v in value.items():
                walk(v, f"{path}.{k}", depth + 1)
        elif isinstance(value, (list, tuple, np.ndarray)) and not isinstance(value, (str, bytes)):
            if len(value) > 0:
                walk(value[0], f"{path}[0]", depth + 1)

    walk(obj, prefix, 0)
    return entries


click_files = list_click_files(CONFIG["clicks_dir"])
print(f"Nombre total de fichiers clicks détectés: {len(click_files)}")
if not click_files:
    print("Aucun fichier clicks trouvé au chemin configuré. Vérifiez le téléchargement des données.")

files_for_eda = click_files[:2]
per_file_stats = []
for file in files_for_eda:
    df_file = pd.read_csv(file)
    ts_col = detect_timestamp_column(df_file)
    article_col = detect_article_column(df_file)
    timestamps = to_timestamp(df_file[ts_col])
    per_file_stats.append(
        {
            "fichier": file.name,
            "nb_lignes": len(df_file),
            "colonnes": ", ".join(df_file.columns),
            "articles_uniques": df_file[article_col].nunique(),
            "horodatage_min": timestamps.min(),
            "horodatage_max": timestamps.max(),
        }
    )
if per_file_stats:
    display(pd.DataFrame(per_file_stats))
else:
    print("Pas assez de fichiers pour réaliser une EDA détaillée par fichier.")

print("=== Clicks (agrégés) ===")
if clicks.empty:
    print("Aucun clic chargé. Vérifier le chemin ou augmenter max_click_files.")
else:
    clicks_summary = {
        "nb_lignes": len(clicks),
        "colonnes": ", ".join(clicks.columns),
        "utilisateurs_uniques": clicks['user_id'].nunique() if 'user_id' in clicks else None,
        "articles_uniques": clicks['article_id'].nunique() if 'article_id' in clicks else None,
    }
    display(pd.DataFrame([clicks_summary]))
    ts_min, ts_max, daily, hourly = summarize_timestamps(clicks['timestamp'])
    display(pd.DataFrame([
        {
            'horodatage_min': ts_min,
            'horodatage_max': ts_max,
            'fenetre_jours': (ts_max - ts_min).days + 1,
        }
    ]))
    print("Répartition par jour (jusqu'à 10 premières valeurs)")
    display(daily.head(10))
    print("Répartition par heure (0-23)")
    display(hourly)

print("=== Métadonnées des articles ===")
if metadata is None:
    print("Aucun fichier metadata chargé.")
else:
    meta_summary = {
        "nb_articles": len(metadata),
        "colonnes": ", ".join(metadata.columns),
        "articles_uniques": metadata['article_id'].nunique() if 'article_id' in metadata else None,
    }
    display(pd.DataFrame([meta_summary]))
    missing = metadata.isna().sum().sort_values(ascending=False)
    display(missing.to_frame('valeurs_manquantes'))
    if 'created_at_ts' in metadata.columns:
        created = to_timestamp(metadata['created_at_ts'])
        display(pd.DataFrame([{'premier_article': created.min(), 'dernier_article': created.max()}]))
    if 'article_id' in metadata.columns:
        overlap = set(clicks['article_id'].unique()) if 'article_id' in clicks.columns else set()
        coverage = len(overlap & set(metadata['article_id'].unique()))
        print(f"Articles présents dans clicks et metadata: {coverage}")


print("=== Embeddings d'articles ===")
embeddings_path = Path(CONFIG['embeddings_path'])
if embeddings_path.exists():
    with embeddings_path.open('rb') as f:
        embeddings_obj = pickle.load(f)
    print(f"Type chargé: {type(embeddings_obj)}")

    def summarize_matrix(mat):
        stats = {
            'shape': getattr(mat, 'shape', None),
            'dtype': getattr(mat, 'dtype', None),
        }

        dim_values = []
        shape = getattr(mat, 'shape', None)
        if shape is not None and len(shape) >= 2:
            dim_values.append(shape[1])
        elif isinstance(mat, (list, tuple, np.ndarray)):
            for row in mat:
                if hasattr(row, '__len__') and not isinstance(row, (str, bytes)):
                    try:
                        dim_values.append(len(row))
                    except TypeError:
                        continue

        if dim_values:
            stats.update({
                'profondeur_min': min(dim_values),
                'profondeur_moyenne': float(np.mean(dim_values)),
                'profondeur_max': max(dim_values),
            })

        if hasattr(mat, 'shape') and len(getattr(mat, 'shape', [])) == 2:
            norms = np.linalg.norm(mat, axis=1)
            stats.update(
                {
                    'nb_vectors': mat.shape[0],
                    'dim': mat.shape[1],
                    'norm_min': norms.min(),
                    'norm_max': norms.max(),
                    'norm_moyenne': norms.mean(),
                }
            )
        return stats

    base_structure = describe_structure(embeddings_obj, max_depth=4)

    if isinstance(embeddings_obj, dict):
        keys = list(embeddings_obj.keys())
        print(f"Clés disponibles: {keys}")
        matrix = embeddings_obj.get('embeddings')
        ids = embeddings_obj.get('articles_ids') or embeddings_obj.get('article_ids')

        structure = base_structure.copy()
        if ids is not None:
            structure.insert(0, {
                'chemin': 'embeddings.article_ids',
                'type': type(ids).__name__,
                'len': len(ids),
                'note': "Identifiants d'articles fournis dans le fichier",
            })
        if structure:
            print("Structure détaillée de l'objet d'embeddings (par chemin de clé):")
            display(pd.DataFrame(structure))

        if matrix is not None:
            stats = summarize_matrix(matrix)
            stats.update(
                {
                    'colonnes': ", ".join(keys),
                    'nb_articles_ids': len(ids) if ids is not None else None,
                    'ids_uniques': len(set(ids)) if ids is not None else None,
                    'couverture_metadata': len(set(ids) & set(metadata['article_id']))
                    if (metadata is not None and ids is not None and 'article_id' in metadata)
                    else None,
                    'couverture_clicks': len(set(ids) & set(clicks['article_id']))
                    if (not clicks.empty and ids is not None and 'article_id' in clicks)
                    else None,
                }
            )
            display(pd.DataFrame([stats]))

            if ids is not None:
                sample_ids = ids[:5] if len(ids) >= 5 else ids
                print("Aperçu des premiers article_id liés aux embeddings:")
                display(pd.DataFrame({'article_id': sample_ids}))

            preview_cols = [f"emb_{i}" for i in range(min(5, matrix.shape[1] if hasattr(matrix, 'shape') else 0))]
            if preview_cols:
                preview = pd.DataFrame(matrix[:5, : len(preview_cols)], columns=preview_cols)
                if ids is not None:
                    preview.insert(0, 'article_id', ids[: len(preview)])
                print("Aperçu des embeddings (quelques colonnes et premières lignes):")
                display(preview)
                print("Colonnes affichées pour l'aperçu des embeddings:")
                print(", ".join(preview.columns))

                if ids is not None and metadata is not None and 'article_id' in metadata:
                    meta_cols = [c for c in ['title', 'category_id', 'created_at_ts', 'publisher'] if c in metadata.columns]
                    meta_sample = (
                        preview[['article_id']]
                        .merge(metadata[['article_id'] + meta_cols], on='article_id', how='left')
                    )
                    if 'created_at_ts' in meta_sample.columns:
                        meta_sample['created_at_ts'] = to_timestamp(meta_sample['created_at_ts'])
                    print("Exemple de liaison embedding -> metadata sur article_id (5 premières lignes):")
                    display(meta_sample.head())
        else:
            print("Aucune matrice d'embeddings explicite trouvée dans l'objet chargé.")
    elif hasattr(embeddings_obj, 'shape'):
        stats = summarize_matrix(embeddings_obj)

        inferred_ids = None
        mapping_note = None
        if metadata is not None and 'article_id' in metadata and hasattr(embeddings_obj, 'shape'):
            if embeddings_obj.shape[0] == len(metadata):
                inferred_ids = metadata['article_id'].reset_index(drop=True)
                mapping_note = (
                    "Aucun article_id explicite fourni ; association supposée alignée sur l'ordre des metadata."
                )
            else:
                mapping_note = (
                    "Aucun article_id dans le fichier d'embeddings et la taille ne correspond pas aux metadata : "
                    f"{embeddings_obj.shape[0]} vecteurs vs {len(metadata)} lignes de metadata."
                )
        else:
            mapping_note = (
                "Aucun identifiant d'article n'est présent dans le fichier d'embeddings (mapping externe requis)."
            )

        structure = base_structure.copy()
        if inferred_ids is not None:
            structure.insert(0, {
                'chemin': 'embeddings.article_id (inféré)',
                'type': type(inferred_ids).__name__,
                'len': len(inferred_ids),
                'note': "Alignement supposé sur metadata.article_id (index identique).",
            })
        if structure:
            print("Structure détaillée de l'objet d'embeddings (par chemin de clé):")
            display(pd.DataFrame(structure))

        if mapping_note:
            print(mapping_note)

        if inferred_ids is not None:
            stats.update(
                {
                    'ids_source': 'metadata.article_id (alignement par index)',
                    'ids_uniques': inferred_ids.nunique(),
                    'couverture_metadata': len(set(inferred_ids) & set(metadata['article_id'])),
                    'couverture_clicks': len(set(inferred_ids) & set(clicks['article_id'])) if not clicks.empty else None,
                }
            )

        display(pd.DataFrame([stats]))
        if len(getattr(embeddings_obj, 'shape', [])) >= 2 and embeddings_obj.shape[1] > 0:
            preview_cols = [f"emb_{i}" for i in range(min(5, embeddings_obj.shape[1]))]
            preview = pd.DataFrame(embeddings_obj[:5, : len(preview_cols)], columns=preview_cols)
            if inferred_ids is not None:
                preview.insert(0, 'article_id', inferred_ids.iloc[: len(preview)].values)
            print("Aperçu direct de la matrice d'embeddings:")
            display(preview)
            print("Colonnes affichées pour l'aperçu des embeddings:")
            print(", ".join(preview.columns))

            if inferred_ids is not None and metadata is not None:
                meta_cols = [c for c in ['title', 'category_id', 'created_at_ts', 'publisher'] if c in metadata.columns]
                meta_sample = preview[['article_id']].merge(
                    metadata[['article_id'] + meta_cols], on='article_id', how='left'
                )
                if 'created_at_ts' in meta_sample.columns:
                    meta_sample['created_at_ts'] = to_timestamp(meta_sample['created_at_ts'])
                print("Exemple de liaison embedding -> metadata sur article_id (inféré):")
                display(meta_sample.head())
        else:
            print("Objet chargé non structuré, utilisez type/len pour investiguer.")
else:
    print(f"Fichier d'embeddings introuvable à {embeddings_path}")




Nombre total de fichiers clicks détectés: 385


,fichier,nb_lignes,colonnes,articles_uniques,horodatage_min,horodatage_max
0,clicks_hour_000.csv,1883,"user_id, session_id, session_start, session_size, click_article_id, click_timestamp, click_environment, click_deviceGroup, click_os, click_country, click_region, click_referrer_type",323,2017-10-01 03:00:00.026,2017-10-03 02:35:54.157
1,clicks_hour_001.csv,1415,"user_id, session_id, session_start, session_size, click_article_id, click_timestamp, click_environment, click_deviceGroup, click_os, click_country, click_region, click_referrer_type",289,2017-10-01 03:36:28.615,2017-10-02 02:41:03.190


=== Clicks (agrégés) ===


,nb_lignes,colonnes,utilisateurs_uniques,articles_uniques
0,31987,"user_id, article_id, timestamp",11642,1826


,horodatage_min,horodatage_max,fenetre_jours
0,2017-10-01 03:00:00.026,2017-10-05 02:26:43.525,4


...

,shape,dtype,profondeur_min,profondeur_moyenne,profondeur_max,nb_vectors,dim,norm_min,norm_max,norm_moyenne,ids_source,ids_uniques,couverture_metadata,couverture_clicks
0,"(364047, 250)",float32,250,250.0,250,364047,250,1.845483,11.18309,7.939456,metadata.article_id (alignement par index),364047,364047,1826


Aperçu direct de la matrice d'embeddings:


,article_id,emb_0,emb_1,emb_2,emb_3,emb_4
0,0,-0.161183,-0.957233,-0.137944,0.050855,0.830055
1,1,-0.523216,-0.974058,0.738608,0.155234,0.626294
2,2,-0.619619,-0.972960,-0.207360,-0.128861,0.044748
3,3,-0.740843,-0.975749,0.391698,0.641738,-0.268645
4,4,-0.279052,-0.972315,0.685374,0.113056,0.238315


Colonnes affichées pour l'aperçu des embeddings:
article_id, emb_0, emb_1, emb_2, emb_3, emb_4
Exemple de liaison embedding -> metadata sur article_id (inféré):


,article_id,category_id,created_at_ts
0,0,0,2017-12-13 05:53:39
1,1,1,2014-07-14 12:45:36
2,2,1,2014-08-22 00:35:06
3,3,1,2014-08-19 17:11:53
4,4,1,2014-08-03 13:06:11


# Article Embeddings

Ce fichier contient les **embeddings des articles**, c’est-à-dire une **représentation numérique du contenu textuel** permettant de comparer les articles entre eux sur le plan sémantique.

* **Format** : matrice NumPy `(N, 250)` en `float32`
* **1 ligne = 1 article**
* **250 colonnes = dimensions latentes**
* Les valeurs individuelles n’ont pas de signification directe

L’`article_id` n’est **pas stocké explicitement** : il est **déduit de l’ordre des lignes**, qui doit rester aligné avec les métadonnées des articles.

La variable `words_count` indique le **nombre de mots du texte source** et sert uniquement d’indicateur de qualité du contenu.

Les embeddings **ne sont pas normalisés** : la **similarité cosinus** est la mesure recommandée pour comparer les articles.


## Protocole

1. Tri des interactions par horodatage pour respecter la chronologie.
2. Split temporel train/test selon `train_ratio` afin d'éviter toute fuite du futur.
3. Construction d'un profil utilisateur à partir des interactions de train.
4. Définition du *ground truth* : articles cliqués en test pour chaque utilisateur (au moins un).
5. Génération de recommandations Top-5 en excluant les articles déjà vus en train.
6. Calcul des métriques de ranking (Precision@5, Recall@5, MAP@5, NDCG@5, Coverage@5) et estimation de la latence moyenne sur un échantillon de 500 utilisateurs max.

Cette démarche imite un scénario de production : d'abord on respecte le temps, puis on mesure simultanément la qualité des suggestions et le coût de calcul.

In [164]:
# Split and utility functions

def temporal_train_test_split(df: pd.DataFrame, train_ratio: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Split interactions chronologically according to the train_ratio."""
    cutoff = int(len(df) * train_ratio)
    train = df.iloc[:cutoff].copy()
    test = df.iloc[cutoff:].copy()
    return train, test


def build_user_histories(df: pd.DataFrame) -> Dict[int, List[int]]:
    """Create mapping user -> list of articles in chronological order."""
    histories: Dict[int, List[int]] = {}
    for user_id, group in df.groupby("user_id"):
        histories[int(user_id)] = group.sort_values("timestamp")["article_id"].tolist()
    return histories


def get_candidate_items(df: pd.DataFrame) -> List[int]:
    """Return unique article ids."""
    return df["article_id"].unique().tolist()


def make_ground_truth(train: pd.DataFrame, test: pd.DataFrame) -> Tuple[Dict[int, List[int]], Dict[int, List[int]]]:
    """Build user histories and ground truth for evaluation.

    Only test items that were seen in training are kept so models are
    evaluated on recommendable articles.
    """
    train_hist = build_user_histories(train)
    candidate_items = set(train["article_id"].unique())
    test_hist = build_user_histories(test)
    filtered = {
        u: [it for it in items if it in candidate_items]
        for u, items in test_hist.items()
        if u in train_hist and len(items) > 0
    }
    eligible_users = {u: items for u, items in filtered.items() if items}
    return train_hist, eligible_users


train_df, test_df = temporal_train_test_split(clicks, CONFIG["train_ratio"])
train_histories, ground_truth = make_ground_truth(train_df, test_df)
candidate_items = get_candidate_items(train_df)
print(f"Train size: {len(train_df)}, Test size: {len(test_df)}, Users for eval: {len(ground_truth)}")


Train size: 25589, Test size: 6398, Users for eval: 317


## Métriques utilisées

* **Precision@5** : part des recommandations top-5 qui sont réellement cliquées (plus c'est haut, plus le Top-5 est précis).
* **Recall@5** : part des clics test retrouvés dans le Top-5 (mesure la couverture de ce que l'utilisateur aime).
* **MAP@5** : moyenne de la précision cumulée à chaque clic retrouvé ; récompense les bonnes positions dans la liste.
* **NDCG@5** : pondère chaque clic par sa position (gain décroissant) et normalise par le meilleur score possible ; idéal pour comparer des classements.
* **Coverage@5** : proportion d'articles différents recommandés sur l'ensemble des utilisateurs (diversité du catalogue).
* **Latence par utilisateur** : temps moyen pour produire le Top-5 (important pour une API temps réel).

In [165]:

# Metrics

def precision_at_k(recommended: List[int], relevant: List[int], k: int) -> float:
    """Precision@k for a single user."""
    if not recommended:
        return 0.0
    rec_k = recommended[:k]
    hits = len(set(rec_k) & set(relevant))
    return hits / k


def recall_at_k(recommended: List[int], relevant: List[int], k: int) -> float:
    """Recall@k for a single user."""
    if not relevant:
        return 0.0
    rec_k = recommended[:k]
    hits = len(set(rec_k) & set(relevant))
    return hits / len(relevant)


def average_precision_at_k(recommended: List[int], relevant: List[int], k: int) -> float:
    """MAP@k for a single user."""
    if not relevant:
        return 0.0
    score = 0.0
    hits = 0
    for i, item in enumerate(recommended[:k], start=1):
        if item in relevant:
            hits += 1
            score += hits / i
    return score / min(len(relevant), k)


def dcg_at_k(recommended: List[int], relevant: List[int], k: int) -> float:
    """Discounted cumulative gain."""
    dcg = 0.0
    for i, item in enumerate(recommended[:k], start=1):
        if item in relevant:
            dcg += 1 / np.log2(i + 1)
    return dcg


def ndcg_at_k(recommended: List[int], relevant: List[int], k: int) -> float:
    """Normalized DCG."""
    ideal_dcg = dcg_at_k(relevant[:k], relevant, k)
    if ideal_dcg == 0:
        return 0.0
    return dcg_at_k(recommended, relevant, k) / ideal_dcg


def coverage_at_k(all_recommendations: List[List[int]], candidate_items: List[int], k: int) -> float:
    """Coverage of unique recommended items over candidates."""
    rec_items = set()
    for rec in all_recommendations:
        rec_items.update(rec[:k])
    if not candidate_items:
        return 0.0
    return len(rec_items) / len(candidate_items)


In [166]:
# Recommenders

def build_global_popularity(train: pd.DataFrame) -> List[int]:
    """Return items sorted by global click counts."""
    return train.groupby("article_id").size().sort_values(ascending=False).index.tolist()


def build_recent_popularity(train: pd.DataFrame, window_days: int) -> List[int]:
    """Return popular items over the last window_days of training data."""
    max_time = train["timestamp"].max()
    window_start = max_time - pd.Timedelta(days=window_days)
    recent = train[train["timestamp"] >= window_start]
    if recent.empty:
        return build_global_popularity(train)
    counts = recent.groupby("article_id")["timestamp"].agg(["size", "max"])
    ranked = counts.sort_values(by=["size", "max"], ascending=[False, False])
    return ranked.index.tolist()


def build_covisit_graph(train: pd.DataFrame) -> Dict[int, Dict[int, int]]:
    """Build co-visitation counts based on user histories."""
    graph: Dict[int, Dict[int, int]] = {}
    for _, group in train.groupby("user_id"):
        items = group.sort_values("timestamp")["article_id"].tolist()
        unique_items = list(dict.fromkeys(items))
        for i, item_i in enumerate(unique_items):
            graph.setdefault(item_i, {})
            for item_j in unique_items[i + 1 :]:
                graph[item_i][item_j] = graph[item_i].get(item_j, 0) + 1
                graph.setdefault(item_j, {})
                graph[item_j][item_i] = graph[item_j].get(item_i, 0) + 1
    return graph


def build_content_embeddings(metadata: pd.DataFrame, pca_components: Optional[int] = None):
    """Create TF-IDF embeddings from textual columns with optional PCA reduction.

    If no free-text columns are present, fall back to using non-ID columns as
    categorical tokens so that content-based similarity remains available.
    """

    text_cols = [
        c
        for c in metadata.columns
        if metadata[c].dtype == object and c not in {"article_id", "clicks"}
    ]
    non_id_cols = [c for c in metadata.columns if c != "article_id"]

    if not text_cols and non_id_cols:
        print("No textual columns in metadata; using non-ID columns as categorical tokens.")
        text_cols = non_id_cols

    if not text_cols:
        raise ValueError("No usable columns in metadata to build content embeddings")

    corpus = metadata[text_cols].fillna("")
    corpus = corpus.apply(lambda row: " ".join(f"{col}_{val}" for col, val in row.items()), axis=1)

    vectorizer = TfidfVectorizer(max_features=5000)
    tfidf = vectorizer.fit_transform(corpus)
    if pca_components and pca_components < tfidf.shape[1]:
        svd = TruncatedSVD(n_components=pca_components, random_state=CONFIG["random_seed"])
        reduced = svd.fit_transform(tfidf)
        embeddings = normalize(reduced)
    else:
        embeddings = normalize(tfidf)
    ids = metadata["article_id"].tolist()
    return embeddings, ids


def build_item_similarity(train: pd.DataFrame, metadata: Optional[pd.DataFrame]):
    """Build item-to-item similarity either from content or co-visitation."""
    if metadata is not None:
        try:
            embeddings, ids = build_content_embeddings(metadata, CONFIG["content_pca_components"])
            similarity: Dict[int, Dict[int, float]] = {}
            for i, aid in enumerate(ids):
                sims = embeddings @ embeddings[i].T
                sims = np.asarray(sims).flatten()
                top_idx = np.argsort(-sims)[1:51]
                similarity[aid] = {ids[j]: float(sims[j]) for j in top_idx if sims[j] > 0}
            return similarity, "content"
        except Exception as exc:
            print(f"Content embeddings failed ({exc}). Falling back to co-visitation.")
    graph = build_covisit_graph(train)
    similarity = {item: {nbr: float(cnt) for nbr, cnt in neigh.items()} for item, neigh in graph.items()}
    return similarity, "covisitation"


def recommend_from_similarity(user_id: int, train_histories: Dict[int, List[int]], similarity: Dict[int, Dict[int, float]], candidate_items: List[int], k: int) -> List[int]:
    """Aggregate similarity scores from user's history."""
    seen = set(train_histories.get(user_id, []))
    scores: Dict[int, float] = {}
    for item in seen:
        for neighbor, sim in similarity.get(item, {}).items():
            if neighbor in seen:
                continue
            scores[neighbor] = scores.get(neighbor, 0.0) + sim
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    recs = [it for it, _ in ranked if it not in seen]
    if len(recs) < k:
        for c in candidate_items:
            if c not in seen and c not in recs:
                recs.append(c)
            if len(recs) >= k:
                break
    return recs[:k]


def build_pure_covisit_similarity(train: pd.DataFrame, top_n_neighbors: int = 100, metric: str = "cosine") -> Dict[int, Dict[int, float]]:
    """Compute pure co-visitation similarity between items.

    The function counts item pairs co-occurring in the same user history,
    then converts counts to either cosine or Jaccard similarity and keeps
    the top-N neighbors per item.
    """

    histories = (
        train.sort_values("timestamp")
        .groupby("user_id")["article_id"]
        .apply(lambda s: list(dict.fromkeys(s.tolist())))
    )
    item_user_counts = train.groupby("article_id")["user_id"].nunique().to_dict()

    co_counts: Dict[int, Dict[int, int]] = {}
    for items in histories:
        for i, item_i in enumerate(items):
            for item_j in items[i + 1 :]:
                if item_i == item_j:
                    continue
                co_counts.setdefault(item_i, {})
                co_counts.setdefault(item_j, {})
                co_counts[item_i][item_j] = co_counts[item_i].get(item_j, 0) + 1
                co_counts[item_j][item_i] = co_counts[item_j].get(item_i, 0) + 1

    similarity: Dict[int, Dict[int, float]] = {}
    for item_i, neighbors in co_counts.items():
        sims = []
        for item_j, count in neighbors.items():
            if metric == "jaccard":
                denom = item_user_counts.get(item_i, 0) + item_user_counts.get(item_j, 0) - count
                if denom <= 0:
                    continue
                sim = count / denom
            else:
                denom = np.sqrt(item_user_counts.get(item_i, 0) * item_user_counts.get(item_j, 0))
                if denom == 0:
                    continue
                sim = count / denom
            sims.append((item_j, float(sim)))
        top_neighbors = sorted(sims, key=lambda x: x[1], reverse=True)[:top_n_neighbors]
        similarity[item_i] = {j: sim for j, sim in top_neighbors}
    return similarity


def build_covisit_recommender(train: pd.DataFrame, top_n_neighbors: int = 100, metric: str = "cosine"):
    """Return a recommend function based purely on co-visitation similarity."""

    similarity = build_pure_covisit_similarity(train, top_n_neighbors=top_n_neighbors, metric=metric)
    popularity = build_global_popularity(train)

    def recommend(user_id: int, seen: set, k: int) -> List[int]:
        scores: Dict[int, float] = {}
        for item in seen:
            for neighbor, sim in similarity.get(item, {}).items():
                if neighbor in seen:
                    continue
                scores[neighbor] = scores.get(neighbor, 0.0) + sim
        ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        recs = [it for it, _ in ranked]
        if len(recs) < k:
            for it in popularity:
                if it not in seen and it not in recs:
                    recs.append(it)
                if len(recs) >= k:
                    break
        return recs[:k]

    meta = {"top_n_neighbors": top_n_neighbors, "metric": metric, "items": len(similarity)}
    return recommend, meta


def build_collaborative_svd(train: pd.DataFrame, n_components: int):
    """Train a simple implicit SVD recommender returning a recommend function."""
    user_codes, user_index = pd.factorize(train["user_id"], sort=True)
    item_codes, item_index = pd.factorize(train["article_id"], sort=True)

    interactions = pd.DataFrame({"user_idx": user_codes, "item_idx": item_codes}).drop_duplicates()
    data = np.ones(len(interactions), dtype=np.float32)
    mat = sparse.coo_matrix((data, (interactions["user_idx"], interactions["item_idx"])), shape=(len(user_index), len(item_index))).tocsr()

    svd = TruncatedSVD(n_components=n_components, random_state=CONFIG["random_seed"])
    user_factors = svd.fit_transform(mat)
    item_factors = svd.components_.T

    user_to_idx = {int(uid): int(idx) for idx, uid in enumerate(user_index.tolist())}
    items = [int(aid) for aid in item_index.tolist()]

    def recommend(user_id: int, seen: set, k: int) -> List[int]:
        if user_id not in user_to_idx:
            popularity = build_global_popularity(train)
            return [it for it in popularity if it not in seen][:k]

        u_vec = user_factors[user_to_idx[user_id]]
        scores = item_factors @ u_vec
        ranked_items = [items[i] for i in np.argsort(-scores)]
        return [it for it in ranked_items if it not in seen][:k]

    meta = {"users": len(user_index), "items": len(item_index), "components": n_components}
    return recommend, meta


def build_hybrid_covisit_recommender(
    train: pd.DataFrame,
    alpha: float,
    top_n_neighbors: int = 100,
    metric: str = "cosine",
):
    """Return a hybrid recommender mixing pure co-visitation and popularity."""

    similarity = build_pure_covisit_similarity(
        train,
        top_n_neighbors=top_n_neighbors,
        metric=metric,
    )
    popularity = build_global_popularity(train)
    popularity_scores = train.groupby("article_id").size()
    max_pop = float(popularity_scores.max()) if not popularity_scores.empty else 0.0
    popularity_scores = (popularity_scores / max_pop).to_dict() if max_pop > 0 else {}

    def recommend(user_id: int, seen: set, k: int) -> List[int]:
        scores: Dict[int, float] = {}
        for item in seen:
            for neighbor, sim in similarity.get(item, {}).items():
                if neighbor in seen:
                    continue
                scores[neighbor] = scores.get(neighbor, 0.0) + alpha * sim
        for item, pop_score in popularity_scores.items():
            if item in seen:
                continue
            scores[item] = scores.get(item, 0.0) + (1 - alpha) * pop_score
        ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        recs = [it for it, _ in ranked]
        if len(recs) < k:
            for it in popularity:
                if it not in seen and it not in recs:
                    recs.append(it)
                if len(recs) >= k:
                    break
        return recs[:k]

    meta = {
        "top_n_neighbors": top_n_neighbors,
        "metric": metric,
        "alpha": alpha,
        "hybrid": True,
        "items": len(similarity),
    }
    return recommend, meta





In [167]:

# Evaluation pipeline

def evaluate_model(
    name: str,
    recommend_func: Callable[[int, set, int], List[int]],
    train_histories: Dict[int, List[int]],
    ground_truth: Dict[int, List[int]],
    candidate_items: List[int],
    k: int,
    latency_sample: int = 500,
) -> Dict[str, float]:
    """Evaluate a recommender with ranking metrics and latency estimation."""
    precisions: List[float] = []
    recalls: List[float] = []
    maps: List[float] = []
    ndcgs: List[float] = []
    all_recs: List[List[int]] = []

    users = list(ground_truth.keys())
    for user_id in users:
        seen = set(train_histories.get(user_id, []))
        recs = recommend_func(user_id, seen, k)
        gt = ground_truth[user_id]
        all_recs.append(recs)
        precisions.append(precision_at_k(recs, gt, k))
        recalls.append(recall_at_k(recs, gt, k))
        maps.append(average_precision_at_k(recs, gt, k))
        ndcgs.append(ndcg_at_k(recs, gt, k))

    coverage = coverage_at_k(all_recs, candidate_items, k)

    sample_users = users[: min(latency_sample, len(users))]
    start = time.perf_counter()
    for user_id in sample_users:
        seen = set(train_histories.get(user_id, []))
        _ = recommend_func(user_id, seen, k)
    latency = (time.perf_counter() - start) / max(1, len(sample_users))

    return {
        "model": name,
        "users": len(users),
        "precision@k": float(np.mean(precisions)),
        "recall@k": float(np.mean(recalls)),
        "map@k": float(np.mean(maps)),
        "ndcg@k": float(np.mean(ndcgs)),
        "coverage@k": coverage,
        "latency_per_user_s": latency,
    }


## Entraînement des systèmes de recommandation

Chaque approche est entraînée séparément pour limiter le temps d'exécution de chaque cellule et mieux contextualiser le rôle de chaque modèle.

### Popularité globale
La recommandation par popularité globale trie les articles par volume d'interactions dans l'ensemble d'entraînement. Elle est rapide à calculer (simple agrégation) et sert de baseline robuste pour comparer les modèles plus avancés.

In [168]:
# Configuration commune
K = CONFIG["k"]

# Popularité globale
popularity_rank = build_global_popularity(train_df)

def popularity_recommender(user_id: int, seen: set, k: int) -> List[int]:
    return [it for it in popularity_rank if it not in seen][:k]

### Popularité récente
Cette variante privilégie la fraîcheur en filtrant les interactions sur une fenêtre temporelle avant de trier les articles par fréquence. Utile pour capter les tendances du moment, au prix d'un recalcul plus fréquent de la fenêtre glissante.

In [169]:
# Popularité récente
recent_rank = build_recent_popularity(train_df, CONFIG["recent_window_days"])

def recent_recommender(user_id: int, seen: set, k: int) -> List[int]:
    return [it for it in recent_rank if it not in seen][:k]

### Collaborative (SVD)
Le filtrage collaboratif factorise la matrice utilisateur-item (SVD) pour capturer des préférences latentes. L'entraînement est plus long que les méthodes de popularité ou de similarité de contenu, mais il modélise mieux les affinités implicites entre utilisateurs et articles.

In [170]:
# Filtrage collaboratif (SVD)
collab_recommend, collab_meta = build_collaborative_svd(train_df, CONFIG["svd_components"])

def collaborative_recommender(user_id: int, seen: set, k: int) -> List[int]:
    return collab_recommend(user_id, seen, k)

In [171]:
# Recommandation co-visitation pure
covisit_recommend_func, covisit_meta = build_covisit_recommender(
    train_df,
    top_n_neighbors=CONFIG["covisit_top_n_neighbors"],
    metric=CONFIG["covisit_similarity"],
)

def covisit_recommender(user_id: int, seen: set, k: int) -> List[int]:
    return covisit_recommend_func(user_id, seen, k)


### Contenu (similarité article-article)
Un modèle basé contenu construit une matrice de similarité entre articles à partir des métadonnées. Les recommandations se font en projetant l'historique utilisateur vers les items proches dans cet espace. Ce calcul peut être plus coûteux car il nécessite la vectorisation et le produit croisé des articles.

In [172]:
# Initialiser un conteneur de résultats pour chaque entraînement
results = []
step_results = []

In [173]:
# Recommandation basée contenu (désactivable)
ENABLE_CONTENT_MODEL = False  # Passer à True pour activer le calcul de similarité contenu

if ENABLE_CONTENT_MODEL:
    item_similarity, sim_mode = build_item_similarity(train_df, metadata)

    def content_recommender(user_id: int, seen: set, k: int) -> List[int]:
        return recommend_from_similarity(user_id, train_histories, item_similarity, candidate_items, k)
else:
    sim_mode = "désactivé"
    content_recommender = None


## Entraînements séparés

Les cinq stratégies sont désormais exécutées dans des cellules distinctes afin de pouvoir lancer, arrêter ou relancer chaque bloc indépendamment. Cela évite d'attendre l'ensemble du pipeline quand un seul entraînement est nécessaire.


### Entraînement 1 : Baseline A – Popularité globale

Cette approche classe les articles par nombre total de clics dans l'historique d'entraînement. Aucun paramètre n'est appris : on calcule simplement le classement global une fois, puis on recommande les articles les plus populaires que l'utilisateur n'a pas encore vus.

In [174]:
popularity_result = evaluate_model(
    "Baseline A - Popularité globale",
    popularity_recommender,
    train_histories,
    ground_truth,
    candidate_items,
    K,
)
results.append(popularity_result)
pd.DataFrame([popularity_result])


,model,users,precision@k,recall@k,map@k,ndcg@k,coverage@k,latency_per_user_s
0,Baseline A - Popularité globale,317,0.102208,0.228077,0.149125,0.201729,0.006298,0.000029


### Entraînement 2 : Baseline B – Popularité récente

On privilégie ici les tendances courantes en recalculant le classement des articles sur la fenêtre temporelle récente (définie par `CONFIG['recent_window_days']`).

In [175]:
recent_result = evaluate_model(
    f"Baseline B - Popularité {CONFIG['recent_window_days']}j",
    recent_recommender,
    train_histories,
    ground_truth,
    candidate_items,
    K,
)
results.append(recent_result)
pd.DataFrame([recent_result])


,model,users,precision@k,recall@k,map@k,ndcg@k,coverage@k,latency_per_user_s
0,Baseline B - Popularité 7j,317,0.102208,0.228077,0.149125,0.201729,0.006298,0.000033


### Entraînement 3 : Modèle C – Item2Item

Le modèle item2item construit une matrice de similarité entre articles (TF-IDF contenu ou co-visitation suivant la disponibilité des métadonnées). Pour chaque utilisateur, on agrège les articles les plus proches de son historique en excluant les items déjà vus.

In [176]:
if not ENABLE_CONTENT_MODEL:
    print("Modèle C - Item2Item désactivé (ENABLE_CONTENT_MODEL=False).")
else:
    item2item_result = evaluate_model(
        f"Modèle C - Item2Item ({sim_mode})",
        content_recommender,
        train_histories,
        ground_truth,
        candidate_items,
        K,
    )
    results.append(item2item_result)
    pd.DataFrame([item2item_result])


Modèle C - Item2Item désactivé (ENABLE_CONTENT_MODEL=False).


### Entraînement 4 : Modèle D – Collaborative SVD

La factorisation de matrice SVD est évaluée après les approches à base de popularité/contenu. On conserve le mécanisme de vérification rapide : si la métrique reste sous le hazard, on stoppe la suite du pipeline pour éviter de consommer du temps sur des configurations mal alignées.


In [177]:
svd_result = evaluate_model(
    "Modèle D - Collaborative SVD",
    collaborative_recommender,
    train_histories,
    ground_truth,
    candidate_items,
    K,
)
results.append(svd_result)

if svd_result["ndcg@k"] < CONFIG["svd_hazard_ndcg"]:
    raise ValueError(
        f"NDCG@{K} du SVD = {svd_result['ndcg@k']:.4f}, sous le hazard {CONFIG['svd_hazard_ndcg']}. Vérifier l'alignement user/item."
    )

pd.DataFrame([svd_result])


,model,users,precision@k,recall@k,map@k,ndcg@k,coverage@k,latency_per_user_s
0,Modèle D - Collaborative SVD,317,0.011356,0.027918,0.020216,0.026798,0.128761,0.000725


### Entraînement 5 : Modèle E – Co-visitation pure

Ce bloc calcule la similarité item-to-item par co-visitation et évalue la recommandation basée sur les co-occurrences.

#### Variantes retenues du modèle E

Nous conservons deux déclinaisons simples :

- **E0** : co-visitation pure.
- **E3** : hybride léger mélangeant E0 et la popularité globale (pondération `alpha`).

Les variantes **E1** (pondération temporelle des co-visites) et **E2** (normalisation lift/PMI pour corriger la popularité brute) ont été écartées : elles n'apportaient pas d'amélioration significative et dégradaient légèrement les métriques par rapport à E0/E3.


In [178]:
covisit_result = evaluate_model(
    f"Modèle E0 - Co-visitation pure ({CONFIG['covisit_similarity']})",
    covisit_recommender,
    train_histories,
    ground_truth,
    candidate_items,
    K,
)
results.append(covisit_result)
step_results.append({**covisit_result, "model": "E0"})
pd.DataFrame([covisit_result])


,model,users,precision@k,recall@k,map@k,ndcg@k,coverage@k,latency_per_user_s
0,Modèle E0 - Co-visitation pure (cosine),317,0.087066,0.193642,0.111642,0.159906,0.150455,0.000042


### Variante E3 : hybride co-visitation + popularité

In [179]:
hybrid_recommend, hybrid_meta = build_hybrid_covisit_recommender(
    train_df,
    alpha=CONFIG["covisit_hybrid_alpha"],
    top_n_neighbors=CONFIG["covisit_top_n_neighbors"],
    metric=CONFIG["covisit_similarity"],
)

def hybrid_covisit_recommender(user_id: int, seen: set, k: int) -> List[int]:
    return hybrid_recommend(user_id, seen, k)


E3_result = evaluate_model(
    f"Modèle E3 - Hybride (alpha={CONFIG['covisit_hybrid_alpha']})",
    hybrid_covisit_recommender,
    train_histories,
    ground_truth,
    candidate_items,
    K,
)
results.append(E3_result)
step_results.append({**E3_result, "model": "E3"})
pd.DataFrame([E3_result])


,model,users,precision@k,recall@k,map@k,ndcg@k,coverage@k,latency_per_user_s
0,Modèle E3 - Hybride (alpha=0.7),317,0.104732,0.233298,0.147469,0.201384,0.055983,0.000324


In [180]:
from __future__ import annotations

from typing import Any, Dict, Callable
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

RANDOM_SEED = 42
N_TRIALS = 1000
BASELINE_LATENCY_S = 0.05  # ajuste si tu as une baseline mesurée

def composite_score(
    eval_result: Dict[str, float],
    baseline_latency_s: float,
    w_ndcg: float = 1.0,
    w_coverage: float = 0.2,
    w_latency: float = 0.05,
) -> float:
    ndcg = float(eval_result["ndcg@k"])
    coverage = float(eval_result["coverage@k"])
    latency = float(eval_result["latency_per_user_s"])
    latency_ratio = latency / max(baseline_latency_s, 1e-9)
    return (w_ndcg * ndcg) + (w_coverage * coverage) - (w_latency * latency_ratio)

def objective(trial: optuna.Trial) -> float:
    alpha = trial.suggest_float("covisit_hybrid_alpha", 0.0, 1.0)
    top_n_neighbors = trial.suggest_int("covisit_top_n_neighbors", 20, 1200, log=True)

    recommend_fn, meta = build_hybrid_covisit_recommender(
        train_df,
        alpha=alpha,
        top_n_neighbors=top_n_neighbors,
        metric=CONFIG["covisit_similarity"],
    )

    eval_result = evaluate_model(
        f"Modèle E3-1 - Hybride optuna (trial {trial.number})",
        recommend_fn,
        train_histories,
        ground_truth,
        candidate_items,
        K,
    )

    # Stocker toutes les infos dans Optuna (évite les listes globales)
    trial.set_user_attr("eval_result", eval_result)
    trial.set_user_attr("meta", meta)

    score = composite_score(eval_result, baseline_latency_s=BASELINE_LATENCY_S)
    trial.report(score, step=0)
    if trial.should_prune():
        raise optuna.TrialPruned()
    return score

sampler = TPESampler(seed=RANDOM_SEED, multivariate=True, group=True)
pruner = MedianPruner(n_startup_trials=50)

study = optuna.create_study(direction="maximize", sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

best_params = study.best_params
best_trial = study.best_trial
best_eval = best_trial.user_attrs["eval_result"]

print("Meilleurs hyperparamètres :", best_params)
print("Score optimisé :", study.best_value)

# Rebuild final model with best params
best_recommend, best_meta = build_hybrid_covisit_recommender(
    train_df,
    alpha=float(best_params["covisit_hybrid_alpha"]),
    top_n_neighbors=int(best_params["covisit_top_n_neighbors"]),
    metric=CONFIG["covisit_similarity"],
)

optimized_result = {**best_eval, "model": "Modèle E3-1 - Hybride optimisé"}
results.append(optimized_result)
step_results.append(optimized_result)

pd.DataFrame([optimized_result])


/home/scude/miniconda3/envs/projet-4/lib/python3.11/site-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
/home/scude/miniconda3/envs/projet-4/lib/python3.11/site-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``group`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-12-17 12:11:01,653] A new study created in memory with name: no-name-b28f9777-4d40-4dd3-8e72-2f64808f5272


  0%|          | 0/1000 [00:00<?, ?it/s]

[I 2025-12-17 12:11:02,069] Trial 0 finished with value: 0.20199602631273203 and parameters: {'covisit_hybrid_alpha': 0.3745401188473625, 'covisit_top_n_neighbors': 980}. Best is trial 0 with value: 0.20199602631273203.
[I 2025-12-17 12:11:02,466] Trial 1 finished with value: 0.21436720819765157 and parameters: {'covisit_hybrid_alpha': 0.7319939418114051, 'covisit_top_n_neighbors': 230}. Best is trial 1 with value: 0.21436720819765157.
[I 2025-12-17 12:11:02,830] Trial 2 finished with value: 0.20117420668141595 and parameters: {'covisit_hybrid_alpha': 0.15601864044243652, 'covisit_top_n_neighbors': 37}. Best is trial 1 with value: 0.21436720819765157.
[I 2025-12-17 12:11:03,307] Trial 3 finished with value: 0.20209487019659772 and parameters: {'covisit_hybrid_alpha': 0.05808361216819946, 'covisit_top_n_neighbors': 692}. Best is trial 1 with value: 0.21436720819765157.
[I 2025-12-17 12:11:03,661] Trial 4 finished with value: 0.20917656047044708 and parameters: {'covisit_hybrid_alpha': 0

,model,users,precision@k,recall@k,map@k,ndcg@k,coverage@k,latency_per_user_s
0,Modèle E3-1 - Hybride optimisé,317,0.106625,0.236574,0.150994,0.205068,0.06648,0.000265


## Résultats consolidés

Après exécution des cinq blocs d'entraînement ci-dessus, les métriques sont agrégées pour comparer les approches. Chaque ligne du tableau récapitule la précision, le rappel, la MAP, le NDCG, la couverture et la latence moyenne par utilisateur.


In [181]:

# Agréger les métriques une fois les entraînements terminés
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(["ndcg@k", "map@k"], ascending=False).reset_index(drop=True)
(display(results_df))


,model,users,precision@k,recall@k,map@k,ndcg@k,coverage@k,latency_per_user_s
0,Modèle E3-1 - Hybride optimisé,317,0.106625,0.236574,0.150994,0.205068,0.066480,0.000265
1,Baseline A - Popularité globale,317,0.102208,0.228077,0.149125,0.201729,0.006298,0.000029
2,Baseline B - Popularité 7j,317,0.102208,0.228077,0.149125,0.201729,0.006298,0.000033
3,Modèle E3 - Hybride (alpha=0.7),317,0.104732,0.233298,0.147469,0.201384,0.055983,0.000324
4,Modèle E0 - Co-visitation pure (cosine),317,0.087066,0.193642,0.111642,0.159906,0.150455,0.000042
5,Modèle D - Collaborative SVD,317,0.011356,0.027918,0.020216,0.026798,0.128761,0.000725


In [182]:
# Comparatif final (baseline vs E3 par défaut vs E3-1 optimisé)
baseline_label = "Baseline A - Popularité globale"
default_e3_label = f"Modèle E3 - Hybride (alpha={CONFIG['covisit_hybrid_alpha']})"
optimized_label = "Modèle E3-1 - Hybride optimisé"

comparison_df = results_df[results_df["model"].isin([baseline_label, default_e3_label, optimized_label])].reset_index(drop=True)
comparison_df


,model,users,precision@k,recall@k,map@k,ndcg@k,coverage@k,latency_per_user_s
0,Modèle E3-1 - Hybride optimisé,317,0.106625,0.236574,0.150994,0.205068,0.066480,0.000265
1,Baseline A - Popularité globale,317,0.102208,0.228077,0.149125,0.201729,0.006298,0.000029
2,Modèle E3 - Hybride (alpha=0.7),317,0.104732,0.233298,0.147469,0.201384,0.055983,0.000324


In [183]:
results_steps = (
    pd.DataFrame(step_results)
    .sort_values(["ndcg@k", "precision@k"], ascending=False)
    .reset_index(drop=True)
)
print("Tableau comparatif des variantes E (trié sur ndcg@k puis precision@k) :")
print(results_steps)


Tableau comparatif des variantes E (trié sur ndcg@k puis precision@k) :
                            model  users  precision@k  recall@k     map@k  \
0  Modèle E3-1 - Hybride optimisé    317     0.106625  0.236574  0.150994   
1                              E3    317     0.104732  0.233298  0.147469   
2                              E0    317     0.087066  0.193642  0.111642   

     ndcg@k  coverage@k  latency_per_user_s  
0  0.205068    0.066480            0.000265  
1  0.201384    0.055983            0.000324  
2  0.159906    0.150455            0.000042  


## Analyse & choix du modèle MVP

Le classement met en lumière des compromis :
- **Pertinence** : la popularité globale obtient le meilleur NDCG@5/MAP@5, signe que trier par volume reste difficile à battre sur ce petit jeu synthétique.
- **Diversité** : l'item2item couvre trois fois plus d'articles, ce qui réduit le risque d'effet tunnel.
- **Latence** : toutes les approches sont très rapides (millisecondes), la popularité restant la plus simple.

Le choix MVP bascule vers la popularité globale uniquement si l'on cherche la pertinence maximale et un déploiement express. Pour un produit, il serait pertinent de tester une hybridation : démarrer par la popularité pour les nouveaux utilisateurs puis basculer vers l'item2item dès que l'historique se construit afin d'augmenter la couverture sans sacrifier la qualité.

In [184]:

best_row = results_df.iloc[0]
justification = f"""
## Choix du modèle MVP

Modèle retenu : **{best_row['model']}**

Motifs principaux :
- NDCG@5 = {best_row['ndcg@k']:.4f}, MAP@5 = {best_row['map@k']:.4f}, Precision@5 = {best_row['precision@k']:.4f}, Recall@5 = {best_row['recall@k']:.4f}
- Couverture = {best_row['coverage@k']:.4f} sur {len(candidate_items)} articles candidats.
- Latence moyenne par utilisateur = {best_row['latency_per_user_s']:.6f} s (CPU).
- Complexité : implémentation {('légère (contenu/co-visitation)' if 'Item2Item' in best_row['model'] else 'linéaire en dimensions SVD')} compatible avec Azure Functions.
- Gestion du cold-start utilisateur via popularité globale.

Note : ajuster `content_pca_components` pour réduire la taille des embeddings en production si nécessaire.
"""
choice_path = Path(CONFIG["artifacts_dir"]) / "model_choice.md"
choice_path.write_text(justification)
print(justification)



## Choix du modèle MVP

Modèle retenu : **Modèle E3-1 - Hybride optimisé**

Motifs principaux :
- NDCG@5 = 0.2051, MAP@5 = 0.1510, Precision@5 = 0.1066, Recall@5 = 0.2366
- Couverture = 0.0665 sur 1429 articles candidats.
- Latence moyenne par utilisateur = 0.000265 s (CPU).
- Complexité : implémentation linéaire en dimensions SVD compatible avec Azure Functions.
- Gestion du cold-start utilisateur via popularité globale.

Note : ajuster `content_pca_components` pour réduire la taille des embeddings en production si nécessaire.



In [185]:

results_path_csv = Path(CONFIG["artifacts_dir"]) / "results.csv"
results_path_json = Path(CONFIG["artifacts_dir"]) / "results.json"
results_df.to_csv(results_path_csv, index=False)
results_df.to_json(results_path_json, orient="records", lines=True)
print(f"Résultats sauvegardés dans {results_path_csv} et {results_path_json}")


Résultats sauvegardés dans ../artifacts/evaluation/results.csv et ../artifacts/evaluation/results.json


## Conclusion

Ce notebook montre comment comparer des stratégies de recommandation avec une procédure reproductible : split temporel, entraînement, évaluation multi-métriques et sauvegarde des résultats. Les essais révèlent que la popularité globale reste une valeur sûre pour débuter, mais que des modèles plus personnalisés (item2item ou SVD) apportent de la diversité dès que l'on dispose d'historique. Les prochaines étapes naturelles sont d'exécuter les tests sur les vraies données Kaggle, d'ajouter des métriques business (taux de clic simulé, couverture par catégorie) et de prototyper une hybridation popularité + item2item dans une Azure Function pour valider le comportement en production.